In [1]:
import sys
import cnmf
import os

# Change path to wherever you have repo locally
sys.path.append('/project/GCRB/Hon_lab/s223695/Data_project/jamboree_2025/cNMF_benchmarking/cNMF_benchmarking_pipeline')

from Inference.src import (
    run_cnmf_consensus, get_top_indices_fast, annotate_genes_to_excel, \
    rename_and_move_files_NMF, rename_all_NMF, compile_results
)

import anndata
import mudata

import pandas as pd

import warnings
warnings.filterwarnings("ignore")



/project/GCRB/Hon_lab/s223695/anaconda3/envs/torch-cNMF/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch

# Check if CUDA is available
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

# Get current GPU details
if torch.cuda.is_available():
    print(f"Current GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Allocated: {torch.cuda.memory_allocated(0)/1024**2:.2f} MB")
    print(f"Memory Reserved: {torch.cuda.memory_reserved(0)/1024**2:.2f} MB")

CUDA Available: True
CUDA Version: 12.4
Number of GPUs: 1
Current GPU: Tesla P40
Memory Allocated: 0.00 MB
Memory Reserved: 0.00 MB


In [19]:
n_iter =10
num_highvar_genes=5451  
components = [30] 
seed = 14
beta_loss = 'frobenius'
init = 'random'
mode = "batch"
algo = 'halsvar'
tol = 1e-7
batch_max_iter = 1000
online_max_pass = 200
online_chunk_size = 50000
online_chunk_max_iter = 500
online_usage_tol = 0.005
online_spectra_tol = 0.005
use_gpu = True
batch_hals_max_iter = 1000
batch_hals_tol = 0.005
sel_thresh = [2.0]
shuffle_cells = True
sk_cd_refit = False

#counts_fn_shuffled = "/oak/stanford/groups/engreitz/Users/ymo/NMF_re-inplementing/Cell_data/shuffled_100K_250genes.h5ad"
counts_fn = "/project/GCRB/Hon_lab/s223695/Data_project/jamboree_2025/data/work_99_304b545580e299107b2429dd55a968_inference_mudata.h5mu"
output_directory = "/project/GCRB/Hon_lab/s223695/Data_project/jamboree_2025/processing_Hon_benchmark/"
run_name = "Honlab_benchmark"

count_adata = "/project/GCRB/Hon_lab/s223695/Data_project/jamboree_2025/data/work_99_304b545580e299107b2429dd55a968_inference_count.h5ad"


In [20]:
adata = mudata.read_h5mu(counts_fn)

In [29]:
adata

MuData object with n_obs × n_vars = 92324 × 9816
  obs:	'batch', 'cov1'
  uns:	'per_element_results', 'per_guide_results'
  3 modalities
    gene:	92324 x 9389
      obs:	'batch', 'cov1', 'batch_number', 'n_counts', 'log1p_n_genes_by_counts', 'total_gene_umis', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'percent_mito', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'num_expressed_genes'
      var:	'symbol', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'gene_chr', 'gene_start', 'gene_end'
    guide:	92324 x 415
      obs:	'batch', 'cov1', 'num_expressed_guides', 'total_guide_umis', 'batch_number'
      var:	'guide_id', 'spacer', 'targeting', 'type', 'guide_chr', 'guide_start', 'guide_end', 'strand', 'PAM', 'intended_target_name', 'intended_target_start', 'intended_target_end', 'gene_name', 'label', 'intended_target_chr'
      uns:	'capture_method', 'moi'
      layers:	'guide_assignment'
    hashing:	92324 x 12
      obs:	'batch', 'cov1', 'cluster_id', 'hto_type', 'hto_type_split'

In [21]:
adata_count = adata["gene"]
adata_count.write(count_adata)

In [22]:

cnmf_obj = cnmf.cNMF(output_dir=output_directory, name=run_name)


In [23]:
cnmf_obj.prepare(counts_fn=count_adata,
                 components=components, n_iter=n_iter, densify=False, tpm_fn=None, num_highvar_genes=num_highvar_genes, genes_file=None,
                init = init,  beta_loss = beta_loss, 
                algo = algo,mode = mode, tol=tol, n_jobs=-1, 
                seed=seed,  use_gpu = use_gpu, 
                alpha_usage=0.0, alpha_spectra=0.0, 
                l1_ratio_usage = 0.0, l1_ratio_spectra = 0.0,
                online_usage_tol=online_usage_tol, online_spectra_tol=online_spectra_tol,
                fp_precision = 'float', 
                batch_max_iter=batch_max_iter, batch_hals_tol = batch_hals_tol, batch_hals_max_iter = batch_hals_max_iter,
                online_max_pass=online_max_pass, online_chunk_size=online_chunk_size, online_chunk_max_iter=online_chunk_max_iter,
                sk_cd_refit=sk_cd_refit, shuffle_cells=shuffle_cells)


input data is shuffled


In [ ]:
cnmf_obj.factorize(total_workers = 1) 

[Worker 0]. Starting task 0.
Use GPU mode.
 niter=10, loss=21636.069143908742.
 niter=20, loss=21629.024943348693.
 niter=30, loss=21626.902135997196.
 niter=40, loss=21625.120207758384.
 niter=50, loss=21624.162781481275.
 niter=60, loss=21623.765814492166.
 niter=70, loss=21623.522747230618.
 niter=80, loss=21623.331843173477.
 niter=90, loss=21623.20975248587.
 niter=100, loss=21623.121328799873.
 niter=110, loss=21623.041784170884.
 niter=120, loss=21622.96260922633.
 niter=130, loss=21622.88343399187.
 niter=140, loss=21622.810178142896.
 niter=150, loss=21622.750981315952.
 niter=160, loss=21622.703993719195.
 niter=170, loss=21622.666625557544.
 niter=180, loss=21622.633327141262.
 niter=190, loss=21622.601138623446.
 niter=200, loss=21622.565990187195.
 niter=210, loss=21622.526031895533.
 niter=220, loss=21622.47830383927.
 niter=230, loss=21622.429095732976.
 niter=240, loss=21622.381737449738.
 niter=250, loss=21622.342148805248.
 niter=260, loss=21622.31254977136.
 niter=27

In [ ]:
cnmf_obj.combine()

In [ ]:
cnmf_obj.k_selection_plot()  

In [ ]:
# Consensus plots with all k to choose thresh

run_cnmf_consensus(cnmf_obj, 
                   components=components, 
                   density_thresholds=sel_thresh)

In [ ]:
# Save all cNMF scores in separate mudata objects
import pandas as pd
import os
import anndata
import muon

for i in sel_thresh:
    for k in K:

        scores = pd.read_csv('{output_directory}/{run_name}/{run_name}.usages.k_{k}.dt_{sel_thresh}.consensus.txt'.format(
                                                                                        output_directory=output_directory,
                                                                                        run_name = run_name,
                                                                                        k=k,
                                                                                        sel_thresh = str(i).replace('.','_')),
                                                                                        sep='\t', index_col=0)

        loadings = pd.read_csv('{output_directory}/{run_name}/{run_name}.spectra.k_{k}.dt_{sel_thresh}.consensus.txt'.format(
                                                                                        output_directory=output_directory,
                                                                                        run_name = run_name,
                                                                                        k=k,
                                                                                        sel_thresh = str(i).replace('.','_')),
                                                                                        sep='\t', index_col=0)
        

        os.makedirs((f'{output_directory}/{run_name}/loading'), exist_ok=True)


        scores.to_csv('{output_directory}/{run_name}/loading/cNMF_scores_{k}_{sel_thresh}.txt'.format(
                                                                                        output_directory=output_directory,
                                                                                        run_name = run_name,
                                                                                        k=k,
                                                                                        sel_thresh = i), sep='\t')
        loadings.T.to_csv('{output_directory}/{run_name}/loading/cNMF_loadings_{k}_{sel_thresh}.txt'.format(     
                                                                                        output_directory=output_directory,
                                                                                        run_name = run_name,
                                                                                        k=k,
                                                                                        sel_thresh = i), sep='\t')

        adata_ = anndata.read_h5ad('{output_directory}/{run_name}/cnmf_tmp/{run_name}.tpm.h5ad'.format(
                                                                                        output_directory=output_directory,
                                                                                        run_name = run_name,
                                                                                        k=k ))
        adata_.var_names_make_unique()
        adata_.obs_names_make_unique()

        prog_data = anndata.AnnData(X=scores.values, obs=adata_.obs)
        prog_data.varm['loadings'] = loadings.values
        prog_data.uns['var_names'] = loadings.columns.values


        # Make adata
        os.makedirs((f'{output_directory}/{run_name}/prog_data'), exist_ok=True)
        prog_data.write(f'{output_directory}/{run_name}/prog_data/NMF_{k}_{sel_thresh}.h5ad'.format(
                                                                                output_directory=output_directory,
                                                                                run_name = run_name,
                                                                                k=k,
                                                                                sel_thresh = str(i).replace('.','_')))

        # Make mdata
        mdata = muon.MuData({'rna': adata_, 'cNMF': prog_data})

        os.makedirs((f'{output_directory}/{run_name}/adata'), exist_ok=True)
        mdata.write(f'{output_directory}/{run_name}/adata/cNMF_{k}_{sel_thresh}.h5mu'.format(
                                                                                output_directory=output_directory,
                                                                                run_name = run_name,
                                                                                k=k,
                                                                                sel_thresh = str(i).replace('.','_')))
    

In [ ]:
# annotation for all K

gene_num = 300
os.makedirs((f'{output_directory}/{run_name}/Annotation'), exist_ok=True)


for i in sel_thresh:
    for k in K:
        df = pd.read_csv('{output_directory}/{run_name}/{run_name}.spectra.k_{k}.dt_{sel_thresh}.consensus.txt'.format(
                                                                                output_directory=output_directory,
                                                                                run_name = run_name,
                                                                                k=k,
                                                                                sel_thresh = str(i).replace('.','_')),
                                                                                sep='\t', index_col=0)   
        overlap = get_top_indices_fast(df, gene_num=gene_num)
        annotate_genes_to_excel(overlap, f'{output_directory}/{run_name}/Annotation/{k}_{i}.xlsx')